In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from timm import create_model
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and transforms
data_dir = "/kaggle/input/breakhist-binary-classificationfew-shot/BreakHist/Final 200X"  # Change to your dataset path
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = ImageFolder(root=f"{data_dir}/Train", transform=transform)
test_dataset = ImageFolder(root=f"{data_dir}/Test", transform=transform)

# Load ViT_small pre-trained on ImageNet-1k
vit_model = create_model('vit_small_patch16_224', pretrained=True)
vit_model.head = nn.Linear(vit_model.num_features, len(train_dataset.classes))  # Add classification head
vit_model.to(device)

# Hyperparameters
num_epochs = 50
learning_rate = 0.001
batch_size = 32
shots_per_class = 5

## Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vit_model.parameters(), lr=learning_rate)

# Prepare support and query sets (only once)
def create_support_and_query(dataset, shots_per_class):
    class_indices = {label: [] for label in range(len(dataset.classes))}
    for idx, (_, label) in enumerate(dataset):
        class_indices[label].append(idx)

    support_indices, query_indices = [], []
    for label, indices in class_indices.items():
        np.random.shuffle(indices)
        support_indices.extend(indices[:shots_per_class])
        query_indices.extend(indices[shots_per_class:])

    return Subset(dataset, support_indices), Subset(dataset, query_indices)

support_set, query_set = create_support_and_query(train_dataset, shots_per_class)

support_loader = DataLoader(support_set, batch_size=1, shuffle=False)
query_loader = DataLoader(query_set, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # Define test_loader

# Feature extraction
def extract_features(data_loader, model):
    features, labels = [], []
    model.eval()
    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Extracting features"):
            images = images.to(device)
            output = model(images).cpu().numpy()
            features.append(output)
            labels.append(targets.numpy())
    return np.vstack(features), np.hstack(labels)

# Training loop with support set only
def train_model_on_support(model, support_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        correct_train = 0
        total_train = 0

        for images, labels in tqdm(support_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

# Train the model on the support set
train_model_on_support(vit_model, support_loader, criterion, optimizer, num_epochs)

# Evaluate on merged query and test dataset
def evaluate_on_merged(model, query_loader, test_loader):
    # Extract features from query set
    query_features, query_labels = extract_features(query_loader, model)
    
    # Extract features from test set
    test_features, test_labels = extract_features(test_loader, model)

    # Combine query and test features and labels
    all_features = np.vstack((query_features, test_features))
    all_labels = np.hstack((query_labels, test_labels))

    # Compute class prototypes from support set
    support_features, support_labels = extract_features(support_loader, model)
    class_prototypes = []
    for label in np.unique(support_labels):
        class_features = support_features[support_labels == label]
        class_prototypes.append(class_features.mean(axis=0))
    class_prototypes = np.array(class_prototypes)

    # Classification using cosine similarity
    correct_merged = 0
    for feature, label in zip(all_features, all_labels):
        similarities = cosine_similarity(feature.reshape(1, -1), class_prototypes)
        probs = torch.softmax(torch.tensor(similarities), dim=1).numpy().flatten()
        predicted_class = np.argmax(probs)
        if predicted_class == label:
            correct_merged += 1

    merged_accuracy = correct_merged / len(all_labels) * 100
    print(f"Accuracy on merged query and test dataset: {merged_accuracy:.4f}%")

# Evaluate final model on the merged dataset
evaluate_on_merged(vit_model, query_loader, test_loader)

model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

Epoch 1/100 - Training: 100%|██████████| 28/28 [00:11<00:00,  2.51it/s]


Epoch [1/100], Loss: 28.7897, Training Accuracy: 75.98%


Extracting features: 100%|██████████| 889/889 [00:06<00:00, 136.53it/s]


Epoch [1/100], Query Accuracy: 59.39%
Best model saved with Query Accuracy: 59.39%


...

Epoch [100/100], Loss: 8.1044, Training Accuracy: 89.27%


Extracting features: 100%|██████████| 889/889 [00:06<00:00, 138.20it/s]
<ipython-input-1-ac5d5dc239ff>:155: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vit_model.load_stat

Epoch [100/100], Query Accuracy: 91.45%


Extracting features: 100%|██████████| 376/376 [00:04<00:00, 84.98it/s]


Accuracy on test dataset: 78.7234%
